In [1]:
import numpy as np
import math
import cv2
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import tools
import pandas as pd
import sys
import os

In [2]:
def boundingBox(pts):
    """
    Arguments:
    ----------
    - `pts`: points of the ellipse
    Return:
    -------
    - min and max coordinates of the points to make the bounding box
    """
    
    ##Output of boundingRect = (x_min, y_min, length of box, width of box):
    rect = cv2.boundingRect(pts)

    ##Get extreme values of coordinates:
    x_min = rect[0]
    y_min = rect[1]
    x_max = rect[0] + rect[2] - 1
    y_max = rect[1] + rect[3] - 1
    
    return (x_min, y_min, x_max, y_max)

In [3]:
##Paths of image and corresponding annotation files:
code_path = '../'
image_file_path = '../images/'
annotation_file_path = '../annotations/'

##Destination directory path of masks:
bb_file_path = '../bounding_boxes/'

##Destination directory
all_im_file_path = '../all_im/'

##Create directory for the bounding boxes text files if it does not already exists:
if not os.path.isdir(bb_file_path):
        os.makedirs(bb_file_path)

        
##Create directory for the file containing all image paths
if not os.path.isdir(all_im_file_path):
        os.makedirs(all_im_file_path)

all_im_fil = all_im_file_path + 'all_im'
        
##Create text file containing paths of all images of the data set        
h = open(all_im_fil,"w+")        

##Loop over all Soccer Ellipse images and process them:
for img_name in os.listdir(image_file_path):
    if img_name[0:8] == 'elps_soc':
        image_path = image_file_path + img_name
        annotation_path = annotation_file_path + img_name[:len(img_name)-3] + 'txt'
        bb_path = bb_file_path + img_name[:len(img_name)-3] + 'txt'
        
        
        ##Write path to image data into text file containing paths of all images
        h.write(image_path + "\n")
        
        
        ##Import image and characteristics:
        original_img = cv2.imread(image_path, cv2.IMREAD_COLOR)
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)
        height = img.shape[0]
        width = img.shape[1]
        
        ##Import the coordinates of all points of the ellipse and process them:
        f = open(annotation_path, "r")
        lines = f.readlines()
        nb_lines = np.shape(lines)
        number_elps = nb_lines[0]

        ##List that will contain the parameters of all bounding boxes
        bb_list = []
        
        
        
        
        ##Compute the bounding box for each ellipse
        for i in range(number_elps):
            current_line = lines[i].split(",")
            
            if current_line == ['\n']:
                break
            
            
            nb_coord = len(current_line) 
            coord_list = []
            
            #print(image_path)
            
            for i in range(nb_coord):
                coord_list.append(float(current_line[i]))

            coord_list = coord_list[: -2]
            coord_array = np.asarray(coord_list)
            points = coord_array.reshape([-1, 2])

            ##Cast coordinates to int type (will be used as coordinates):
            pts = points.astype(int)

            ##Conversion from Cytomine coordinates system to opencv coordinates system:
            for i in range(len(pts)):
                pts[i][1] = height - pts[i][1]
    
            x_min, y_min, x_max, y_max = boundingBox(pts)
            bb = []
            bb.append(x_min)
            bb.append(y_min)
            bb.append(x_max)
            bb.append(y_max)
            bb_list.append(bb)

            
        
        f.close()

        
        g = open(bb_path,"w+")
        
        if current_line == ['\n']:
            g.write("\n")
            g.close()
            continue
        
            
            
        
        for i in range(number_elps):
            x_min = bb_list[i][0]
            y_min = bb_list[i][1]
            x_max = bb_list[i][2]
            y_max = bb_list[i][3]
        
            xc_box = ((x_max+x_min)/2)/width
            yc_box = ((y_max+y_min)/2)/height
            wid_box = (x_max-x_min)/width
            hei_box = (y_max-y_min)/height
            if i >= 1:
                g.write("\n")
            g.write("0 " + str(xc_box) + " " + str(yc_box) + " " + str(wid_box) + " " + str(hei_box))
        
        g.close() 
        
        
        
        
        
          

h.close()
print("All BB text files created")

All BB text files created
